In [70]:
from PIL import Image
import shutil
import cv2 as cv
import os
from sklearn.utils import shuffle
import torch , torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader , Dataset
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
from torch import optim
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

plt.ion()

%matplotlib inline

In [71]:
class_new = ['anger', 'surprise', 'disgust', 'neutral', 'happiness',
       'sadness']

In [72]:
class_new_val = {'anger':0, 'surprise':1, 'disgust':2, 'neutral':3, 'happiness':4,
       'sadness':5}

In [73]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.ToPILImage(),
        transforms.Resize([128,128]),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomHorizontalFlip(),                                
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5],std=[0.5]),
    ])

In [74]:
class ImageDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
    
  def __len__(self):
    return len(self.frame)
  
  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir,self.frame.iloc[idx, 0])
    image = Image.open(img_name)
    sample = [ image, self.frame.iloc[idx,1]]
    if self.transform:
      sample[0] = self.transform(sample[0])
    return sample

In [75]:
face_dataset = ImageDataset(csv_file='new.csv' , root_dir='images_dataset' , transform = data_transform)
for i in range(len(face_dataset)):
    sample = face_dataset[i]
    print(i, sample[0].shape, sample[1])
    if i == 10:
        plt.show()
        break

0 torch.Size([1, 128, 128]) 0
1 torch.Size([1, 128, 128]) 5
2 torch.Size([1, 128, 128]) 2
3 torch.Size([1, 128, 128]) 2
4 torch.Size([1, 128, 128]) 1
5 torch.Size([1, 128, 128]) 3
6 torch.Size([1, 128, 128]) 5
7 torch.Size([1, 128, 128]) 4
8 torch.Size([1, 128, 128]) 0
9 torch.Size([1, 128, 128]) 3
10 torch.Size([1, 128, 128]) 0


In [76]:
num_workers = 0
batch_size = 30
valid_size = 0.2
test_size = 0.1
num_train = len(face_dataset)
indices = list(range(num_train))
np.random.shuffle(indices)
valid_set = int(np.floor(valid_size * num_train))
test_set  = int(np.floor(test_size * num_train))
test_idx,valid_idx , train_idx = indices[:test_set], indices[test_set:test_set+valid_set] , indices[test_set+valid_set :] 
                                                                                                   


In [77]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)


train_loader = torch.utils.data.DataLoader(face_dataset, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(face_dataset, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(face_dataset, batch_size=batch_size, 
    num_workers=num_workers)

In [78]:

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


In [79]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 500)
        self.fc2 = nn.Linear(500, 6)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = Net()
# model.cuda()
print(model)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=6, bias=True)
  (dropout): Dropout(p=0.25)
)


In [80]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [81]:
model_new = Net()
model_new.load_state_dict(torch.load("model_augmented.pt"))
model_new.eval()

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=6, bias=True)
  (dropout): Dropout(p=0.25)
)

In [82]:
print("Model's state_dict:")
for param_tensor in model_new.state_dict():
    print(param_tensor, "\t", model_new.state_dict()[param_tensor].size())

print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([16, 1, 3, 3])
conv1.bias 	 torch.Size([16])
conv2.weight 	 torch.Size([32, 16, 3, 3])
conv2.bias 	 torch.Size([32])
conv3.weight 	 torch.Size([64, 32, 3, 3])
conv3.bias 	 torch.Size([64])
fc1.weight 	 torch.Size([500, 16384])
fc1.bias 	 torch.Size([500])
fc2.weight 	 torch.Size([6, 500])
fc2.bias 	 torch.Size([6])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139636312820688, 139636312820832, 139636312820760, 139636312821120, 139636312818024, 139636312821552, 139636312818456, 139636312817736, 139636312817952, 139636312818744]}]


In [83]:
test_loss = 0.0
class_correct = list(0. for i in range(6))
class_total = list(0. for i in range(6))

model_new.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model_new(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        


test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(6):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% ' % (
            class_new[i], 100 * class_correct[i] / class_total[i]))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (class_new[i]))

print('\nTest Accuracy (Overall): %2d%%' % (
    100. * np.sum(class_correct) / np.sum(class_total)))

Test Loss: 1.053354

Test Accuracy of anger: 67% 
Test Accuracy of surprise: 65% 
Test Accuracy of disgust: 67% 
Test Accuracy of neutral: 43% 
Test Accuracy of happiness: 71% 
Test Accuracy of sadness: 53% 

Test Accuracy (Overall): 61%
